# Homework and bake-off: Relation extraction using distant supervision

In [ ]:
__author__ = "Bill MacCartney and Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Baselines](#Baselines)
  1. [Hand-build feature functions](#Hand-build-feature-functions)
  1. [Distributed representations](#Distributed-representations)
1. [Homework questions](#Homework-questions)
  1. [Different model factory [1 points]](#Different-model-factory-[1-points])
  1. [Directional unigram features [1.5 points]](#Directional-unigram-features-[1.5-points])
  1. [The part-of-speech tags of the "middle" words [1.5 points]](#The-part-of-speech-tags-of-the-"middle"-words-[1.5-points])
  1. [Bag of Synsets [2 points]](#Bag-of-Synsets-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

This homework and associated bake-off are devoted to the developing really effective relation extraction systems using distant supervision. 

As with the previous assignments, this notebook first establishes a baseline system. The initial homework questions ask you to create additional baselines and suggest areas for innovation, and the final homework question asks you to develop an original system for you to enter into the bake-off.

## Set-up

See [the first notebook in this unit](rel_ext_01_task.ipynb#Set-up) for set-up instructions.

In [1]:
import numpy as np
import os
import rel_ext
from sklearn.linear_model import LogisticRegression
import utils

As usual, we unite our corpus and KB into a dataset, and create some splits for experimentation:

In [2]:
rel_ext_data_home = os.path.join('data', 'rel_ext_data')

In [3]:
corpus = rel_ext.Corpus(os.path.join(rel_ext_data_home, 'corpus.tsv.gz'))

In [78]:
corpus.get_examples_for_entities('Randall_Munroe','xkcd')

[]

In [4]:
kb = rel_ext.KB(os.path.join(rel_ext_data_home, 'kb.tsv.gz'))

In [5]:
dataset = rel_ext.Dataset(corpus, kb)

You are not wedded to this set-up for splits. The bake-off will be conducted on a previously unseen test-set, so all of the data in `dataset` is fair game:

In [6]:
splits = dataset.build_splits(
    split_names=['tiny', 'train', 'dev'],
    split_fracs=[0.01, 0.79, 0.20],
    seed=1)

In [7]:
splits

{'tiny': Corpus with 3,474 examples; KB with 445 triples,
 'train': Corpus with 263,285 examples; KB with 36,191 triples,
 'dev': Corpus with 64,937 examples; KB with 9,248 triples,
 'all': Corpus with 331,696 examples; KB with 45,884 triples}

## Baselines

### Hand-build feature functions

In [170]:
def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        #print(kbt.sbj,kbt.obj,kbt.rel,ex)
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter

In [171]:
featurizers = [simple_bag_of_words_featurizer]

In [172]:
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

In [173]:
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.853      0.391      0.690        340       5716
author                    0.765      0.544      0.708        509       5885
capital                   0.529      0.189      0.390         95       5471
contains                  0.791      0.601      0.744       3904       9280
film_performance          0.797      0.564      0.736        766       6142
founders                  0.740      0.397      0.631        380       5756
genre                     0.510      0.147      0.342        170       5546
has_sibling               0.857      0.253      0.580        499       5875
has_spouse                0.866      0.327      0.651        594       5970
is_a                      0.693      0.227      0.492        497       5873
nationality               0.533      0.163      0.366        301       5677
parents     

In [56]:
len(baseline_results['vectorizer'].get_feature_names())

29654

Studying model weights might yield insights:

In [39]:
rel_ext.examine_model_weights(baseline_results)

Highest and lowest feature weights for relation adjoins:

     2.537 Córdoba
     2.452 Valais
     2.421 Taluks
     ..... .....
    -1.113 his
    -1.156 had
    -2.264 Earth

Highest and lowest feature weights for relation author:

     3.251 author
     2.911 book
     2.800 books
     ..... .....
    -1.998 chapter
    -2.068 or
    -2.889 Booker

Highest and lowest feature weights for relation capital:

     3.269 capital
     1.849 posted
     1.684 km
     ..... .....
    -1.513 Westminster
    -1.558 includes
    -1.679 borough

Highest and lowest feature weights for relation contains:

     2.580 bordered
     2.162 southwestern
     2.146 third-largest
     ..... .....
    -2.542 film
    -3.076 Midlands
    -3.891 Ceylon

Highest and lowest feature weights for relation film_performance:

     3.991 starring
     3.967 alongside
     3.810 co-starring
     ..... .....
    -2.004 She
    -2.047 spy
    -2.097 then

Highest and lowest feature weights for relation founders:

  

### Distributed representations

This simple baseline sums the GloVe vector representations for all of the words in the "middle" span and feeds those representations into the standard `LogisticRegression`-based `model_factory`. The crucial parameter that enables this is `vectorize=False`. This essentially says to `rel_ext.experiment` that your featurizer or your model will do the work of turning examples into vectors; in that case, `rel_ext.experiment` just organizes these representations by relation type.

In [16]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

In [17]:
glove_lookup = utils.glove2dict(
    os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

In [191]:
def glove_middle_featurizer(kbt, corpus, np_func=np.sum):
    #print("inside function")
    reps = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        #print("inside for loop")
        for word in ex.middle.split():
            #print("Word: ", word)
            rep = glove_lookup.get(word)
            #print("Rep: ", rep)
            if rep is not None:
                reps.append(rep)
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    if len(reps) == 0:
        #print("inside if condition")
        dim = len(next(iter(glove_lookup.values())))
        #print("dim" ,dim)
        return utils.randvec(n=dim)
    else:
        return np_func(reps, axis=0)

In [19]:
glove_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[glove_middle_featurizer],    
    vectorize=False, # Crucial for this featurizer!
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.841      0.482      0.732        340       5716
author                    0.816      0.444      0.699        509       5885
capital                   0.541      0.211      0.412         95       5471
contains                  0.655      0.412      0.586       3904       9280
film_performance          0.841      0.330      0.642        766       6142
founders                  0.769      0.245      0.538        380       5756
genre                     0.478      0.065      0.210        170       5546
has_sibling               0.786      0.251      0.551        499       5875
has_spouse                0.877      0.347      0.671        594       5970
is_a                      0.699      0.159      0.416        497       5873
nationality               0.656      0.209      0.460        301       5677
parents     

In [192]:
from collections import defaultdict
kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
feature_counter = defaultdict(int)
print(feature_counter)
# Make sure `feature_counter` is being updated, not reinitialized:
feature_counter = glove_middle_featurizer(kbt, corpus)
feature_counter

defaultdict(<class 'int'>, {})


array([-4.59439888e-01, -3.72354492e-01, -2.60069040e-01,  4.83596541e-01,
       -2.89197201e-01, -3.42550297e-01, -3.00892603e-01, -1.27534672e-01,
       -1.88834058e-01,  9.35937850e-02, -3.98623569e-02,  7.41134448e-02,
        2.33621850e-01,  1.26951844e-01, -4.68364859e-01, -3.59116000e-01,
        1.59745695e-01,  1.29638072e-02, -2.87449956e-01,  3.98327753e-01,
        1.02752529e-01,  2.44152902e-02,  1.36152919e-02,  4.14719507e-01,
        2.09707509e-01,  2.32410485e-01,  1.92412916e-01,  5.89693967e-02,
       -3.44321233e-02,  7.95951234e-02,  4.04517592e-01,  6.49960961e-02,
       -3.93675820e-01, -1.99960387e-01,  2.79875964e-01, -2.62433156e-01,
       -2.94416219e-01, -3.20878791e-01,  2.52426627e-01,  2.53331153e-01,
        3.89780241e-01,  1.33012162e-01,  3.26709789e-01, -2.68887694e-01,
        1.42139295e-01, -1.21682221e-02,  1.90077197e-01,  4.06096016e-01,
       -3.46672046e-01,  4.66040051e-01,  1.28311290e-01, -3.83373436e-01,
       -5.50703976e-02,  

With the same basic code design, one can also use the PyTorch models included in the course repo, or write new ones that are better aligned with the task. For those models, it's likely that the featurizer will just return a list of tokens (or perhaps a list of lists of tokens), and the model will map those into vectors using an embedding.

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Different model factory [1 points]

The code in `rel_ext` makes it very easy to experiment with other classifier models: one need only redefine the `model_factory` argument. This question asks you to assess a [Support Vector Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

__To submit:__ A wrapper function `run_svm_model_factory` that does the following: 

1. Uses `rel_ext.experiment` with the model factory set to one based in an `SVC` with `kernel='linear'` and all other arguments left with default values. 
1. Trains on the 'train' part of `splits`.
1. Assesses on the `dev` part of `splits`.
1. Uses `featurizers` as defined above. 
1. Returns the return value of `rel_ext.experiment` for this set-up.

The function `test_run_svm_model_factory` will check that your function conforms to these general specifications.

In [44]:
def run_svm_model_factory():
    from sklearn.svm import LinearSVC
    model_factory_svm = lambda: LinearSVC(C=1)
    svm_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory_svm,
    verbose=True)
    return svm_results



In [45]:
def test_run_svm_model_factory(run_svm_model_factory):
    results = run_svm_model_factory()
    assert 'featurizers' in results, \
        "The return value of `run_svm_model_factory` seems not to be correct"
    # Check one of the models to make sure it's an SVC:
    assert 'SVC' in results['models']['adjoins'].__class__.__name__, \
        "It looks like the model factor wasn't set to use an SVC."    

In [46]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_svm_model_factory(run_svm_model_factory)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.725      0.356      0.600        340       5716
author                    0.700      0.582      0.672        509       5885
capital                   0.431      0.263      0.382         95       5471
contains                  0.754      0.620      0.722       3904       9280
film_performance          0.725      0.607      0.698        766       6142
founders                  0.643      0.432      0.586        380       5756
genre                     0.393      0.259      0.356        170       5546
has_sibling               0.718      0.244      0.517        499       5875
has_spouse                0.819      0.350      0.646        594       5970
is_a                      0.520      0.294      0.450        497       5873
nationality               0.400      0.199      0.333        301       5677
parents     

### Directional unigram features [1.5 points]

The current bag-of-words representation makes no distinction between "forward" and "reverse" examples. But, intuitively, there is big difference between _X and his son Y_ and _Y and his son X_. This question asks you to modify `simple_bag_of_words_featurizer` to capture these differences. 

__To submit:__

1. A feature function `directional_bag_of_words_featurizer` that is just like `simple_bag_of_words_featurizer` except that it distinguishes "forward" and "reverse". To do this, you just need to mark each word feature for whether it is derived from a subject–object example or from an object–subject example.  The included function `test_directional_bag_of_words_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `directional_bag_of_words_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

3. `rel_ext.experiment` returns some of the core objects used in the experiment. How many feature names does the `vectorizer` have for the experiment run in the previous step? Include the code needed for getting this value. (Note: we're partly asking you to figure out how to get this value by using the sklearn documentation, so please don't ask how to do it!)

In [81]:
def directional_bag_of_words_featurizer(kbt, corpus, feature_counter): 
    # Append these to the end of the keys you add/access in 
    # `feature_counter` to distinguish the two orders. You'll
    # need to use exactly these strings in order to pass 
    # `test_directional_bag_of_words_featurizer`.
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"
    
    ##### YOUR CODE HERE

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        #print(kbt.sbj,kbt.obj,kbt.rel,ex)
        for word in ex.middle.split(' '):
            feature_counter[word+subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word+object_subject_suffix] += 1
    return feature_counter

featurizers = [directional_bag_of_words_featurizer]

# Call to `rel_ext.experiment`:
##### YOUR CODE HERE    
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)



relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.860      0.435      0.720        340       5716
author                    0.821      0.603      0.766        509       5885
capital                   0.548      0.242      0.437         95       5471
contains                  0.816      0.670      0.782       3904       9280
film_performance          0.824      0.661      0.785        766       6142
founders                  0.781      0.413      0.663        380       5756
genre                     0.621      0.241      0.472        170       5546
has_sibling               0.842      0.246      0.568        499       5875
has_spouse                0.894      0.355      0.686        594       5970
is_a                      0.742      0.266      0.546        497       5873
nationality               0.625      0.216      0.453        301       5677
parents     

In [75]:
def test_directional_bag_of_words_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['is_OS'] += 5
    feature_counter = directional_bag_of_words_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'is_OS':6,'a_OS':1,'webcomic_OS':1,'created_OS':1,'by_OS':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [76]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_directional_bag_of_words_featurizer(corpus)

In [79]:
len(baseline_results['vectorizer'].get_feature_names())

1591

In [ ]:
baseline_results['vectorizer'].get_feature_names()

### The part-of-speech tags of the "middle" words [1.5 points]

Our corpus distribution contains part-of-speech (POS) tagged versions of the core text spans. Let's begin to explore whether there is information in these sequences, focusing on `middle_POS`.

__To submit:__

1. A feature function `middle_bigram_pos_tag_featurizer` that is just like `simple_bag_of_words_featurizer` except that it creates a feature for bigram POS sequences. For example, given 

  `The/DT dog/N napped/V`
  
   we obtain the list of bigram POS sequences
  
   `b = ['<s> DT', 'DT N', 'N V', 'V </s>']`. 
   
   Of course, `middle_bigram_pos_tag_featurizer` should return count dictionaries defined in terms of such bigram POS lists, on the model of `simple_bag_of_words_featurizer`.  Don't forget the start and end tags, to model those environments properly! The included function `test_middle_bigram_pos_tag_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `middle_bigram_pos_tag_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

In [233]:
def middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter): 
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        #print(kbt.sbj,kbt.obj,kbt.rel,ex)
        pos_list = get_tags(ex.middle_POS)
        #print(pos_list)
        bigram_list = get_tag_bigrams(pos_list)
        #print(bigram_list)
        for word in bigram_list:
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        pos_list = get_tags(ex.middle_POS)
        bigram_list = get_tag_bigrams(pos_list)
        for word in bigram_list:
            feature_counter[word] += 1
    return feature_counter


def get_tag_bigrams(s):
    """Suggested helper method for `middle_bigram_pos_tag_featurizer`.
    This should be defined so that it returns a list of str, where each 
    element is a POS bigram."""
    # The values of `start_symbol` and `end_symbol` are defined
    # here so that you can use `test_middle_bigram_pos_tag_featurizer`.
    start_symbol = "<s>"
    end_symbol = "</s>"
    seq = [start_symbol] + s + [end_symbol]
    #print(seq)
    bigrams = [i+" "+j for i,j in zip(seq[:-1],seq[1:])]
    return bigrams
    ##### YOUR CODE HERE

def get_tags(s): 
    """Given a sequence of word/POS elements (lemmas), this function
    returns a list containing just the POS elements, in order.    
    """
    return [parse_lem(lem)[1] for lem in s.strip().split(' ') if lem]

def parse_lem(lem):
    """Helper method for parsing word/POS elements. It just splits
    on the rightmost / and returns (word, POS) as a tuple of str."""
    return lem.strip().rsplit('/', 1)  

featurizers = [middle_bigram_pos_tag_featurizer]

# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.791      0.356      0.636        340       5716
author                    0.702      0.338      0.578        509       5885
capital                   0.417      0.158      0.314         95       5471
contains                  0.765      0.594      0.723       3904       9280
film_performance          0.713      0.446      0.637        766       6142
founders                  0.542      0.171      0.378        380       5756
genre                     0.652      0.176      0.424        170       5546
has_sibling               0.618      0.168      0.403        499       5875
has_spouse                0.760      0.266      0.554        594       5970
is_a                      0.520      0.183      0.380        497       5873
nationality               0.412      0.070      0.208        301       5677
parents     

In [234]:
def test_middle_bigram_pos_tag_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['<s> VBZ'] += 5
    feature_counter = middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'<s> VBZ':6,'VBZ DT':1,'DT JJ':1,'JJ VBN':1,'VBN IN':1,'IN </s>':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [235]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_middle_bigram_pos_tag_featurizer(corpus)

### Bag of Synsets [2 points]

The following allows you to use NLTK's WordNet API to get the synsets compatible with _dog_ as used as a noun:

```
from nltk.corpus import wordnet as wn
dog = wn.synsets('dog', pos='n')
dog
[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01')]
```

This question asks you to create synset-based features from the word/tag pairs in `middle_POS`.

__To submit:__

1. A feature function `synset_featurizer` that is just like `simple_bag_of_words_featurizer` except that it returns a list of synsets derived from `middle_POS`. Stringify these objects with `str` so that they can be `dict` keys. Use `convert_tag` (included below) to convert tags to `pos` arguments usable by `wn.synsets`. The included function `test_synset_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `synset_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment`.)

In [165]:
from nltk.corpus import wordnet as wn

def synset_featurizer(kbt, corpus, feature_counter):
    #print(feature_counter)
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        #print(kbt.sbj,kbt.obj,kbt.rel,ex)
        pos_list = get_synsets(ex.middle_POS)
        #print(pos_list)
        #bigram_list = get_tag_bigrams(pos_list)
        #print(bigram_list)
        for word in pos_list:
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        pos_list = get_synsets(ex.middle_POS)
        #print(pos_list)
        #bigram_list = get_tag_bigrams(pos_list)
        for word in pos_list:
            feature_counter[word] += 1
    return feature_counter


    return feature_counter


def get_synsets(s):
    """Suggested helper method for `synset_featurizer`. This should
    be completed so that it returns a list of stringified Synsets 
    associated with elements of `s`.
    """   
    # Use `parse_lem` from the previous question to get a list of
    # (word, POS) pairs. Remember to convert the POS strings.
    synset_list = []
    wt = [parse_lem(lem) for lem in s.strip().split(' ') if lem]
    #print("pairs:" , wt)
    for pair in wt:
        pos_wn_format = convert_tag(pair[1])
        #print("new pos:" , pos_wn_format)
        if pos_wn_format != None:
            #print("inside if condition")
            for element in wn.synsets(pair[0],pos = pos_wn_format):
                synset_list.append(str(element))
        #print("fetched synsets: " , synset_list)
    return synset_list
    ##### YOUR CODE HERE


    
    
def convert_tag(t):
    """Converts tags so that they can be used by WordNet:
    
    | Tag begins with | WordNet tag |
    |-----------------|-------------|
    | `N`             | `n`         |
    | `V`             | `v`         |
    | `J`             | `a`         |
    | `R`             | `r`         |
    | Otherwise       | `None`      |
    """        
    if t[0].lower() in {'n', 'v', 'r'}:
        return t[0].lower()
    elif t[0].lower() == 'J':
        return 'a'
    else:
        return None    

featurizers = [synset_featurizer]

# Call to `rel_ext.experiment`:
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)   




relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.724      0.309      0.571        340       5716
author                    0.749      0.328      0.596        509       5885
capital                   0.564      0.232      0.438         95       5471
contains                  0.716      0.408      0.622       3904       9280
film_performance          0.781      0.479      0.693        766       6142
founders                  0.720      0.358      0.599        380       5756
genre                     0.413      0.112      0.268        170       5546
has_sibling               0.809      0.212      0.518        499       5875
has_spouse                0.888      0.308      0.645        594       5970
is_a                      0.613      0.191      0.425        497       5873
nationality               0.500      0.130      0.318        301       5677
parents     

In [158]:
def test_synset_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter["Synset('be.v.01')"] += 5
    feature_counter = synset_featurizer(kbt, corpus, feature_counter)
    # The full return values for this tend to be long, so we just
    # test a few examples to avoid cluttering up this notebook.
    test_cases = {
        "Synset('be.v.01')": 6,
        "Synset('embody.v.02')": 1
    }
    for ss, expected in test_cases.items():
        print("ss: ", ss)
        print("Expected:" ,expected)
        result = feature_counter[ss]
        print("results: ",result)
        assert result == expected, \
            "Incorrect count for {}: Expected {}; Got {}".format(ss, expected, result)

In [159]:
abc = [['is', 'VBZ'], ['a', 'DT'], ['webcomic', 'JJ'], ['created', 'VBN'], ['by', 'IN']]
for a in abc:
    print(a[1])
    convert_tag(a[1])

VBZ
DT
JJ
VBN
IN


In [160]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_synset_featurizer(corpus)

defaultdict(<class 'int'>, {"Synset('be.v.01')": 5})
pairs: [['is', 'VBZ'], ['a', 'DT'], ['webcomic', 'JJ'], ['created', 'VBN'], ['by', 'IN']]
new pos: v
inside if condition
fetched synsets:  ["Synset('be.v.01')", "Synset('be.v.02')", "Synset('be.v.03')", "Synset('exist.v.01')", "Synset('be.v.05')", "Synset('equal.v.01')", "Synset('constitute.v.01')", "Synset('be.v.08')", "Synset('embody.v.02')", "Synset('be.v.10')", "Synset('be.v.11')", "Synset('be.v.12')", "Synset('cost.v.01')"]
new pos: None
fetched synsets:  ["Synset('be.v.01')", "Synset('be.v.02')", "Synset('be.v.03')", "Synset('exist.v.01')", "Synset('be.v.05')", "Synset('equal.v.01')", "Synset('constitute.v.01')", "Synset('be.v.08')", "Synset('embody.v.02')", "Synset('be.v.10')", "Synset('be.v.11')", "Synset('be.v.12')", "Synset('cost.v.01')"]
new pos: None
fetched synsets:  ["Synset('be.v.01')", "Synset('be.v.02')", "Synset('be.v.03')", "Synset('exist.v.01')", "Synset('be.v.05')", "Synset('equal.v.01')", "Synset('constitute.v.0

### Your original system [3 points]

There are many options, and this could easily grow into a project. Here are a few ideas:

- Try out different classifier models, from `sklearn` and elsewhere.
- Add a feature that indicates the length of the middle.
- Augment the bag-of-words representation to include bigrams or trigrams (not just unigrams).
- Introduce features based on the entity mentions themselves. <!-- \[SPOILER: it helps a lot, maybe 4% in F-score. And combines nicely with the directional features.\] -->
- Experiment with features based on the context outside (rather than between) the two entity mentions — that is, the words before the first mention, or after the second.
- Try adding features which capture syntactic information, such as the dependency-path features used by Mintz et al. 2009. The [NLTK](https://www.nltk.org/) toolkit contains a variety of [parsing algorithms](http://www.nltk.org/api/nltk.parse.html) that may help.
- The bag-of-words representation does not permit generalization across word categories such as names of people, places, or companies. Can we do better using word embeddings such as [GloVe](https://nlp.stanford.edu/projects/glove/)?

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies.

In [ ]:
# Enter your system description in this cell.
# Please do not remove this comment.



## Bake-off [1 point]

For the bake-off, we will release a test set. The announcement will go out on the discussion forum. You will evaluate your custom model from the previous question on these new datasets using the function `rel_ext.bake_off_experiment`. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

The cells below this one constitute your bake-off entry.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [241]:
# Enter your bake-off assessment code in this cell. 
# Please do not remove this comment.
#if 'IS_GRADESCOPE_ENV' not in os.environ:
    # Please enter your code in the scope of the above conditional.
    ##### YOUR CODE HERE

from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
subject_object_suffix = "_SO"
object_subject_suffix = "_OS"

def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        #print(kbt.sbj,kbt.obj,kbt.rel,ex)
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter

def directional_bag_of_words_featurizer(kbt, corpus, feature_counter): 
    # Append these to the end of the keys you add/access in 
    # `feature_counter` to distinguish the two orders. You'll
    # need to use exactly these strings in order to pass 
    # `test_directional_bag_of_words_featurizer`.
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"
    
    ##### YOUR CODE HERE

    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        #print(kbt.sbj,kbt.obj,kbt.rel,ex)
        for word in ex.middle.split(' '):
            feature_counter[word+subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word+object_subject_suffix] += 1
    return feature_counter

def directional_bag_of_words_featurizer_middle_left_right(kbt, corpus, feature_counter):
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        #print(kbt.sbj,kbt.obj,kbt.rel,ex)
        for word in ex.middle.split(' '):
            feature_counter[word+subject_object_suffix] += 1
        for word in ex.left.split(' '):
            feature_counter[word+subject_object_suffix] += 1
        for word in ex.right.split(' '):
            feature_counter[word+subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word+object_subject_suffix] += 1
        for word in ex.left.split(' '):
            feature_counter[word+object_subject_suffix] += 1
        for word in ex.right.split(' '):
            feature_counter[word+object_subject_suffix] += 1
    return feature_counter

def directional_bag_of_words_featurizer_middle_left_right(kbt, corpus, feature_counter):
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        #print(kbt.sbj,kbt.obj,kbt.rel,ex)
        for word in ex.middle.split(' '):
            feature_counter[word+subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word+object_subject_suffix] += 1
    return feature_counter

def middle_bigram_featurizer(kbt, corpus, feature_counter): 
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        #print(kbt.sbj,kbt.obj,kbt.rel,ex)
        word_list = get_middle_bigrams(ex.middle)
        #print(word_list)
        for word in word_list:
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        word_list = get_middle_bigrams(ex.middle)
        for word in word_list:
            feature_counter[word] += 1
    return feature_counter


def get_middle_bigrams(s):
    """Suggested helper method for `middle_bigram_pos_tag_featurizer`.
    This should be defined so that it returns a list of str, where each 
    element is a POS bigram."""
    # The values of `start_symbol` and `end_symbol` are defined
    # here so that you can use `test_middle_bigram_pos_tag_featurizer`.
    start_symbol = "<s> "
    end_symbol = " </s>"
    seq = start_symbol + s + end_symbol
    #print("middle sendtence: ", seq)
    seq_list = seq.split(' ')
    #print("middle sentence list", seq_list)
    bigrams = [i for i in zip(seq_list[:-1],seq_list[1:])]
    return bigrams

def middle_length_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        #print(kbt.sbj,kbt.obj,kbt.rel,ex)
        feature_counter['LENGTH_S_O'] = len(ex.middle.split(' '))
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        feature_counter['LENGTH_O_S'] = len(ex.middle.split(' '))
    return feature_counter

def left_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.left.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.left.split(' '):
            feature_counter[word] += 1
    return feature_counter

def right_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.right.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.right.split(' '):
            feature_counter[word] += 1
    return feature_counter

def dir_left_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.left.split(' '):
            feature_counter[word+subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.left.split(' '):
            feature_counter[word+object_subject_suffix] += 1
    return feature_counter

def dir_right_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.right.split(' '):
            feature_counter[word+subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.right.split(' '):
            feature_counter[word+object_subject_suffix] += 1
    return feature_counter

In [244]:
# Call to `rel_ext.experiment`:
##### YOUR CODE HERE    
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[simple_bag_of_words_featurizer,
                directional_bag_of_words_featurizer,
                 middle_bigram_pos_tag_featurizer,
                 left_bag_of_words_featurizer,
                 right_bag_of_words_featurizer,
                 middle_length_featurizer,
                #dir_left_bag_of_words_featurizer,
                #dir_right_bag_of_words_featurizer
                ],
    model_factory=lambda: LogisticRegression(fit_intercept=True, solver='liblinear'),
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.820      0.509      0.731        340       5716
author                    0.824      0.792      0.817        509       5885
capital                   0.528      0.295      0.456         95       5471
contains                  0.840      0.829      0.838       3904       9280
film_performance          0.860      0.735      0.831        766       6142
founders                  0.807      0.561      0.742        380       5756
genre                     0.632      0.282      0.506        170       5546
has_sibling               0.859      0.697      0.821        499       5875
has_spouse                0.827      0.643      0.782        594       5970
is_a                      0.765      0.577      0.719        497       5873
nationality               0.734      0.641      0.713        301       5677
parents     

In [ ]:
from collections import defaultdict
kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
feature_counter = defaultdict(int)
print(feature_counter)
# Make sure `feature_counter` is being updated, not reinitialized:
feature_counter = middle_bigram_pos_tag_featurizer(kbt, corpus,feature_counter)
feature_counter

In [245]:
# On an otherwise blank line in this cell, please enter
# your macro-average f-score (an F_0.5 score) as reported 
# by the code above. Please enter only a number between 
# 0 and 1 inclusive. Please do not remove this comment.
#if 'IS_GRADESCOPE_ENV' not in os.environ:
    # Please enter your score in the scope of the above conditional.
    ##### YOUR CODE HERE
